In [8]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint

Podaci se sastoje od 48x48 sivih slika lica.
Slike su unaprijed pridodjeljene jednoj od 7 mogućih kategorija (emocija) podijeljenjih na indexe od 0 do 6 - (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)

In [4]:
# Load the dataset and split it into training and validation sets
data = pd.read_csv("./archive/fer2013.csv")

train_data = data[data["Usage"] == "Training"]
test_data = data[data["Usage"] == "PrivateTest"]

x_train = train_data["pixels"].values
y_train = to_categorical(train_data["emotion"].values)

x_test = test_data["pixels"].values
y_test = to_categorical(test_data["emotion"].values)

In [5]:
# Define the CNN model
# dropout layer prevents overfitting
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(48, 48, 1), activation="relu"))
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(7, activation="softmax"))

In [6]:
# Preprocess the images
x_train = np.array([np.fromstring(x, dtype=int, sep=" ").reshape((48, 48, 1)) for x in x_train])
x_test = np.array([np.fromstring(x, dtype=int, sep=" ").reshape((48, 48, 1)) for x in x_test])
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [11]:
checkpointer = ModelCheckpoint('static/model.h5', save_best_only=True)
# Compile and fit the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), 
                    steps_per_epoch=len(x_train) / 32, 
                    epochs=3,
                    validation_data=datagen.flow(x_test, y_test, batch_size=32),
                    callbacks=[checkpointer]
            )

Epoch 1/3
897/897 [==============================] - 96s 107ms/step - loss: 1.1003 - accuracy: 0.5762 - val_loss: 1.2580 - val_accuracy: 0.5252
Epoch 2/3
 72/897 [=>............................] - ETA: 1:31 - loss: 1.0015 - accuracy: 0.6228

KeyboardInterrupt: 